# Créer une ressource de données de fichier URI
Une ressource de données de fichier URI pointe vers un fichier spécifique. Azure Machine Learning stocke uniquement le chemin d’accès au fichier, ce qui signifie que vous pouvez pointer vers n’importe quel type de fichier. Lorsque vous utilisez la ressource de données, vous spécifiez la façon dont vous souhaitez lire les données, ce qui dépend du type de données auquel vous vous connectez.

Les chemins d’accès pris en charge que vous pouvez utiliser lors de la création d’une ressource de données de fichier URI sont les suivants :

Local : ./<path>
Stockage Blob Azure : wasbs://<account_name>.blob.core.windows.net/<container_name>/<folder>/<file>
Azure Data Lake Storage (Gen 2) : abfss://<file_system>@<account_name>.dfs.core.windows.net/<folder>/<file>
Magasin de données : azureml://datastores/<datastore_name>/paths/<folder>/<file>

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_path = '<supported-path>'

my_data = Data(
    path=my_path,
    type=AssetTypes.URI_FILE,
    description="<description>",
    name="<name>",
    version="<version>"
)

ml_client.data.create_or_update(my_data)

Lorsque vous analysez la ressource de données de fichier URI en tant qu’entrée dans un travail Azure Machine Learning, vous devez d’abord lire les données avant de pouvoir les utiliser.

Imaginez que vous créez un script Python que vous souhaitez exécuter en tant que travail et que vous définissez la valeur du paramètre d’entrée input_data comme étant la ressource de données de fichier URI (qui pointe vers un fichier CSV). Vous pouvez lire les données en incluant le code suivant dans votre script Python :

In [ ]:
import argparse
import pandas as pd

parser = argparse.ArgumentParser()
parser.add_argument("--input_data", type=str)
args = parser.parse_args()

df = pd.read_csv(args.input_data)
print(df.head(10))

Si votre ressource de données de fichier URI pointe vers un autre type de fichier, vous devez utiliser le code Python approprié pour lire les données. Par exemple, si au lieu de fichiers CSV, vous utilisez des fichiers JSON, vous devriez plutôt utiliser pd.read_json().

## Créer une ressource de données de dossier URI
Une ressource de données de `dossier URI`pointe vers un dossier spécifique. Elle fonctionne de la même manière qu’une ressource de données de fichier URI et prend en charge les mêmes chemins.

Pour créer une ressource de données de dossier URI avec le SDK Python, vous pouvez utiliser le code suivant

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_path = '<supported-path>'

my_data = Data(
    path=my_path,
    type=AssetTypes.URI_FOLDER,
    description="<description>",
    name="<name>",
    version='<version>'
)

ml_client.data.create_or_update(my_data)

Lorsque vous analysez la ressource de données de dossier URI en tant qu’entrée dans un travail Azure Machine Learning, vous devez d’abord lire les données avant de pouvoir les utiliser.

Imaginez que vous créez un script Python que vous souhaitez exécuter en tant que travail et que vous définissez la valeur du paramètre d’entrée input_data comme étant la ressource de données de dossier URI (qui pointe vers plusieurs fichiers CSV). Vous pouvez vouloir lire tous les fichiers CSV dans le dossier et les concaténer, ce que vous pouvez faire en incluant le code suivant dans votre script Python :

In [ ]:
import argparse
import glob
import pandas as pd

parser = argparse.ArgumentParser()
parser.add_argument("--input_data", type=str)
args = parser.parse_args()

data_path = args.input_data
all_files = glob.glob(data_path + "/*.csv")
df = pd.concat((pd.read_csv(f) for f in all_files), sort=False)

## Créer une ressource de données MLTable
Une ressource de `données MLTable`vous permet de pointer vers des données tabulaires. Lorsque vous créez une ressource de données MLTable, vous spécifiez la définition du schéma pour lire les données. Comme le schéma est déjà défini et stocké avec la ressource de données, vous n’avez pas besoin de spécifier comment lire les données lorsque vous l’utilisez.

Par conséquent, utilisez une ressource de données MLTable lorsque le schéma de vos données est complexe ou change fréquemment. Au lieu de modifier la façon de lire les données dans chaque script qui utilise les données, il vous suffit de les modifier dans la ressource de données elle-même.

Lorsque vous définissez le schéma durant la création d’une ressource de données MLTable, vous pouvez également choisir de spécifier uniquement un sous-ensemble des données.

Pour certaines fonctionnalités d’Azure Machine Learning, comme le Machine Learning automatisé, vous devez utiliser une ressource de données MLTable, car Azure Machine Learning doit savoir comment lire les données.

Pour définir le schéma, il est recommandé d’inclure un fichier MLTable dans le même dossier que les données que vous souhaitez lire. Le fichier MLTable inclut le chemin d’accès pointant vers les données que vous souhaitez lire et la façon de lire les données :
```yaml
type: mltable

paths:
  - pattern: ./*.txt
transformations:
  - read_delimited:
      delimiter: ','
      encoding: ascii
      header: all_files_same_headers
```


Pour créer une ressource de données MLTable avec le SDK Python, vous pouvez utiliser le code suivant :

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_path = '<path-including-mltable-file>'

my_data = Data(
    path=my_path,
    type=AssetTypes.MLTABLE,
    description="<description>",
    name="<name>",
    version='<version>'
)

ml_client.data.create_or_update(my_data)

Lorsque vous analysez une ressource de données MLTable en tant qu’entrée dans un script Python que vous souhaitez exécuter comme travail Azure Machine Learning, vous pouvez inclure le code suivant pour lire les données :

In [ ]:
import argparse
import mltable
import pandas

parser = argparse.ArgumentParser()
parser.add_argument("--input_data", type=str)
args = parser.parse_args()

tbl = mltable.load(args.input_data)
df = tbl.to_pandas_dataframe()

print(df.head(10))


Une approche courante consiste à convertir les `données tabulaires en trame de données Pandas`. Cependant, vous pouvez également convertir les `données en trame de données Spark`si cela convient mieux à votre charge de travail.

## Exercice - Rendre les données disponibles

À présent, vous avez la possibilité d’explorer comment utiliser des données.

Dans cet exercice, vous allez :

- Explorez les magasins de données par défaut.
- Créez une banque de données.
- Créez des ressources de données.

Instructions
Suivez ces instructions pour effectuer l’exercice :

Consultez le référentiel d’exercices sur https://microsoftlearning.github.io/mslearn-azure-ml/.
Faites l’exercice Rendre les données disponibles dans Azure Machine Learning.

```python
 pip uninstall azure-ai-ml
 pip install azure-ai-ml
 pip install mltable
```